# 3.1 Example - QEXAFS Data Processing - Data Inport, Cleanup, Process Correlation

Goal of Workflow:

    1. Import all QEXAFS data
    2. Calculate absorption coefficients
    3. Remove Beamline data leaving only energy and mu
    4. Import process data
    5. Correlate QEXAFS and process data
    6. Save data for further processing
    
### Steps:
    1. Import Modules
    2. Define Paths
    3. Define Beamline Data Structure
    4. Add Data to Experiment Object
    5. Clean up Data
    6. Calculate Absorption Spectra
    7. Remove Raw Data from Experiment
    8. Import Process Data Streams
    9. Correlate Process Parameters to QEXAFS Data
    10. Save Experiment with 'pickle'

### Things to have ready when attemtping this example:
    1. Directoty containing only XAS spectra
    2. Data structure of the XAS files
    3. Directory and filename where to save the experiment object (pickling)
    4. Directoy and filenames of process data streams (mass spec. and LabView)

## Step 1. Import Modules

In [1]:
# DO NOT MODIFY BELOW

# File Handling
import os
import pickle

# Make Plotting interactive
from ipywidgets import interact, fixed

# Wrapper for xraylarch (larch)
os.chdir('..')
pwd = os.getcwd()
os.chdir('./catxas')
import general as fcts
import xas as xfcts
import plot as pfcts
import process
import experiment as exp


# Plot Handling
%matplotlib inline

# Time Elapsed counting
import time

start_time = time.process_time()

## Step 2. Define Paths

#### Modify:
    1. data_dir - directory containing XAS spetra ONLY!
    2. pickle_dir - directory where to save the experiment
    3. pickpe_name - filename to save the experiment (must include extension '.pickle')
    4. MS_fname - full path, filename, and extension to mass spectrometer file
    5. LV_fname - full path, filename, and extension to LabView file

In [2]:
# Folder containing ONLY XAS spectra
data_dir = r'D:\UU QEXAFS Workup\Nina - BF31\Fe Edge\Up\20211119_ME_BF31_Fe_B_UP'

# Directory where to save the experiment:
pickle_dir = r'D:\UU QEXAFS Workup\Nina - BF31\Fe Edge\Up\Results\Step B'

# File name to save the sxperiment (must include extension '.pickle')
pickle_name = 'BF31-Fe-B'

# Path and file name of mass spectrometer file
MS_fname = r'D:\UU QEXAFS Workup\Nina - BF31\Process Parameter Files\20211119_ME_BF31View4.csv'

# Path and file name of LabView file
LV_fname = r'D:\UU QEXAFS Workup\Nina - BF31\Process Parameter Files\T02R0035-19112021-Process.txt'

## Step 3. Define Beamline Data Structure

Common data formats for SSRL beamlines can be found in the "BL specific XAS data structures.ipynb" notebook.

In [3]:
### Define the layout of the xas data file and how spectra are calcualted
xas_data_structure = {
    'time stamp': True,
    'time on line': 6,
    'time format': '# First Scan Create Date:\t%m/%d/%Y %I:%M:%S %p ',
    'padded scan numbers': True,
    'column names': ['Encoder', 'Energy', 'ADC_01', 'ADC_02', 'ADC_03', 'ADC_04', 'ADC_05', 'Time', 'Gate'],
    'energy column' : 'Energy', # Energy [eV]
    'sample numerator': 'ADC_01', # I0
    'sample denominator': 'ADC_02', # I1
    'sample ln': True,
    'sample invert': False,
    'reference numerator': 'ADC_01', # I0
    'reference denominator': 'ADC_03', # I2
    'reference ln': True,
    'reference invert': False,
    'is QEXAFS': True
}

## Step 4. Add Data to Experiment Object

    The Experiment class is a dictionary-like obect that is used to store XAS data, process data streams, and correlation and fitting results.
    
### Major steps for adding data to the Experiment class:
    a. Create Experiment Class
    b. Load XAS data into the Experiment object
    

### 4a. Create Experiment Class

#### MODIFY:
    1. exp_name - unique identifier for the experiment object

In [4]:
# Uniquie name for the experiment
exp_name = 'BF31-Fe-B'

In [5]:
# DO NOT MODIFY

# Create the Experimental Class
my_exp = exp.Experiment(exp_name)

### 4b. Load XAS data into the Experiment object

In [6]:
# Add raw data into spectra files
my_exp.import_spectra_data(data_dir, xas_data_structure, print_name = False)

elapsed_time = time.process_time()-start_time
print(f'Time to import data: {elapsed_time/60} minutes')

Time to import data: 9.340885416666667 minutes


## Step 5. Clean up Data
#### STILL IN DEVELOPMENT
    If spectra were collected using CXAS or QXAS there is a chance that the raw data may need cleaning up to remove
    duplicate data points or nan/inf values.
    
    for QXAS, data parsing may result in short/long data sets that need to be removed prior to further processing.
    
### 5a. Remove duplicate points, NaN/inf values, and Zero values

In [7]:
my_exp.organize_RawData(remove_duplicates=True, remove_nan_inf=False, remove_zeros=False) 
# remove_nan_inf & remove_zeros does not work yet

Duplicate data points removed
Range of data points per raw spectra: 357-40021
Range of data points per duplicates removed spectra: 210-37174


### 5b. Remove short/long data sets

In [8]:
# Determine and summarize the short/long data sets
bad_spectra = my_exp.data_length_screen(deviations = 3)

S̲p̲e̲c̲t̲r̲a̲ ̲D̲a̲t̲a̲ ̲L̲e̲n̲g̲h̲t̲ ̲C̲h̲a̲r̲a̲c̲t̲e̲r̲i̲s̲t̲i̲c̲s̲:
	Spectra interrogated: 2546
	Longest Data Set: 37174 data points
	Shortest Data Set: 210 data points
	Mean Data Points per Spectrum: 19315
	Deviation in Data Points: 545


P̲r̲o̲b̲l̲e̲m̲a̲t̲i̲c̲ ̲S̲p̲e̲c̲t̲r̲a̲:
	Number of Spectra: 3
		20211119_ME_BF31_Fe_B_00001
                      10860
		20211119_ME_BF31_Fe_B_03998
                      37174
		20211119_ME_BF31_Fe_B_04199
                        210


In [9]:
# Remove the bad spectra from the dataset
my_exp.remove_bad_spectra(bad_spectra['Filename'])

Removing 20211119_ME_BF31_Fe_B_00001
	Removed from Spectra
	Removed from the summary: "XAS Spectra Files
Removing 20211119_ME_BF31_Fe_B_03998
	Removed from Spectra
	Removed from the summary: "XAS Spectra Files
Removing 20211119_ME_BF31_Fe_B_04199
	Removed from Spectra
	Removed from the summary: "XAS Spectra Files
Spectra Removed list has been created in Summary


## Step 6. Calculate Absorption Spectra

#### MODIFY:
    1. sample_spectra - True = calcualtes sample mux, False = does not calcualte sample mux
    2. ref_spectra - True = calcualtes reference mux, False = does not calcualte reference mux

In [10]:
# Calcualte mux for Sample:
sample_spectra = True

# Calcualte mux for Reference:
ref_spectra = True

In [11]:
# DO NOT MODIFY BELOW

my_exp.calculate_spectra(sample_spectra = sample_spectra, ref_spectra = ref_spectra)

## Step 7. Remove BL data from experimental class

In [12]:
# DO NOT MODIFY BELOW
my_exp.remove_BLData()

Beamline data removed from experiment


## Step 8. Add Process Data to Experiment 

    Note: Additiona datastreams will need their own functions. Please reach out with questions.

In [13]:
# Add Mass Spec Dataset
my_exp.import_massspec(MS_fname)

# Add LabView Dataset
my_exp.import_labview(LV_fname)

## Step 9. Correlate Process Parameters to XAS data

    This step interpolates values from the parameter datasets onto the XAS spectra time grid.

In [14]:
# DO NOT MODIFY
my_exp.correlate_process_params()

Genearted Spectra Summary
Merged Index for Process: MS Data
Merged Index for Process: LV Data


## Step 10. Save Experiment with 'pickle' 
    Pickle - A method to save the experimental class and reopen it elsewhere.

In [15]:
pickle_path = os.path.join(pickle_dir,pickle_name+'.pickle')
pickle_out = open(pickle_path,"wb")
pickle.dump(my_exp, pickle_out)
pickle_out.close()

# Do not forget to kill this kernal to prevent pileup on the RAM